## Visualizing DEICODE feature ranks with rankratioviz
In this example, we use data from [this Qiita study](https://qiita.ucsd.edu/study/description/10422). It's associated with the following paper:

Tripathi, A., Melnik, A. V., Xue, J., Poulsen, O., Meehan, M. J., Humphrey, G., ... & Haddad, G. (2018). Intermittent hypoxia and hypercapnia, a hallmark of obstructive sleep apnea, alters the gut microbiome and metabolome. _mSystems, 3_(3), e00020-18.

### Requirements
This notebook relies on QIIME 2, DEICODE, q2-emperor, and rankratioviz all being installed.

### Setting up
In this section, we clear the output directory and then import our BIOM table as a `FeatureTable[Frequency]` QIIME 2 artifact.

In [1]:
# Clear the output directory so we can write these files there
!rm -rf output/*
# Since git doesn't keep track of empty directories, create the output/ directory if it doesn't already exist
# (if it does already exist, -p ensures that an error won't be thrown)
!mkdir -p output

In [2]:
!qiime tools import \
    --input-path input/qiita_10422_table.biom \
    --output-path output/qiita_10422_table.biom.qza \
    --type FeatureTable[Frequency]

Imported input/qiita_10422_table.biom as BIOMV210DirFmt to output/qiita_10422_table.biom.qza


### Running DEICODE through QIIME 2
Now, we run DEICODE on our imported BIOM table. This produces two output files: a biplot and a distance matrix. (We're going to use rankratioviz to visualize the feature rankings, which are contained in the biplot output file.)

Please see [DEICODE's official documentation](https://library.qiime2.org/plugins/deicode) for more information about how it works and how its output files are formatted.

In [3]:
!qiime deicode rpca \
    --i-table output/qiita_10422_table.biom.qza \
    --o-biplot output/ordination.qza \
    --o-distance-matrix output/dist_matrix.qza

Saved PCoAResults % Properties(['biplot']) to: output/ordination.qza
Saved DistanceMatrix to: output/dist_matrix.qza


### Optional: Visualizing the DEICODE biplot in [Emperor](https://docs.qiime2.org/2019.1/plugins/available/emperor/)
This step isn't required if you just want to use DEICODE with rankratioviz. However, it provides some interesting context about the biplot that DEICODE just generated.

To quote the DEICODE documentation linked above:
> Biplots are exploratory visualization tools that allow us to represent the features (i.e. taxonomy or OTUs) that strongly influence the principal component axis as arrows. The interpretation of the compositional biplot differs slightly from classical biplot interpretation [...] The important features with regard to sample clusters are not a single arrow but [...] the log ratio between features represented by arrows pointing in different directions.

In [4]:
!qiime emperor biplot \
    --i-biplot output/ordination.qza \
    --m-sample-metadata-file input/qiita_10422_metadata.tsv \
    --m-feature-metadata-file input/taxonomy.tsv \
    --o-visualization output/biplot.qzv \
    --p-number-of-features 8

Saved Visualization to: output/biplot.qzv


As we'll show below, rankratioviz will help us understand these "log ratios" mentioned above in more detail.

### Using rankratioviz through QIIME 2
You can use rankratioviz inside or outside of QIIME 2. First, let's show how to use rankratioviz from within QIIME 2. Since we're working with DEICODE output, we use the `qiime rankratioviz unsupervised-rank-plot` command.

In [5]:
!qiime rankratioviz unsupervised-rank-plot --help

Usage: qiime rankratioviz unsupervised-rank-plot [OPTIONS]

  Generates an interactive visualization of DEICODE ranks in tandem with a
  visualization of the log ratios of selected ranks' sample abundance.

Options:
  --i-ranks ARTIFACT PATH PCoAResults % Properties(['biplot'])
                                  A ordination file describing ranks produced
                                  by DEICODE  [required]
  --i-table ARTIFACT PATH FeatureTable[Frequency]
                                  A BIOM table describing the abundances of
                                  the ranked featuresin samples.  [required]
  --m-sample-metadata-file MULTIPLE FILE
                                  Metadata file or artifact viewable as
                                  metadata. This option may be supplied
                                  multiple times to merge metadata.
                                  [required]
  --m-feature-metadata-file MULTIPLE FILE
                                  Metadata 

In [6]:
!qiime rankratioviz unsupervised-rank-plot \
    --i-ranks output/ordination.qza \
    --i-table output/qiita_10422_table.biom.qza \
    --m-sample-metadata-file input/qiita_10422_metadata.tsv \
    --m-feature-metadata-file input/taxonomy.tsv \
    --o-visualization output/rrv_plot_q2.qzv

Saved Visualization to: output/rrv_plot_q2.qzv


That's it! Now, we've created a QZV file (describing a rankratioviz visualization) at `output/rrv_plot_q2.qzv`. You can view this visualization in one of the following ways:
  1. Upload the QZV file to [view.qiime2.org](https://view.qiime2.org).
  2. View the QZV file using `qiime tools view`.

### Using rankratioviz as a standalone tool
We don't need to use rankratioviz through QIIME 2; if you want, you can run it by itself. The benefit to this is that you don't need to install QIIME 2 to run rankratioviz this way.

Here, we just export the `ordination.qza` file we already generated using `qiime deicode rpca` to an `ordination.txt` file. This file can be understood by rankratioviz when run outside of QIIME 2.

(In practice, you can also generate a `RPCA_ordination.txt` file -- which can be read by rankratioviz outside of QIIME 2 -- by running `deicode` outside of QIIME 2.)

In [7]:
# export ordination to a .txt file
!qiime tools export \
    --input-path output/ordination.qza \
    --output-path output

Exported output/ordination.qza as OrdinationDirectoryFormat to directory output


In [8]:
!rankratioviz --help

Usage: rankratioviz [OPTIONS]

  Generates a plot of ranked taxa/metabolites and their abundances.

Options:
  -r, --ranks TEXT              Differentials output from songbird or
                                Ordination output from DEICODE.  [required]
  -t, --table TEXT              BIOM table describing taxon/metabolite sample
                                abundances.  [required]
  -fm, --feature-metadata TEXT  Feature metadata file.
  -sm, --sample-metadata TEXT   Sample metadata file.  [required]
  -o, --output-dir TEXT         Location of output files.  [required]
  --help                        Show this message and exit.


In [9]:
!rankratioviz \
    --ranks output/ordination.txt \
    --table input/qiita_10422_table.biom \
    --sample-metadata input/qiita_10422_metadata.tsv \
    --feature-metadata input/taxonomy.tsv \
    --output-dir output/rrv_plot_standalone/

All sample(s) in the sample metadata file were supported in the BIOM table.
Successfully generated a visualization in the folder output/rrv_plot_standalone/.


We just generated a rankratioviz visualization in the folder `output/rrv_plot_standalone/`. This visualization is analogous to the QZV file we generated above using QIIME 2. You can view this visualization by just opening up `output/rrv_plot_standalone/index.html` in a modern web browser.

That's it! If you have any more questions about using rankratioviz, feel free to contact us (see the rankratioviz README for contact information).